In [1]:
from datetime import datetime as dt
from typing import Iterable

from dateutil import parser
from pystac import Collection, Item, MediaType
from pystac_client import Client, ItemSearch
from pystac_client.stac_api_io import StacApiIO
from urllib3 import Retry

In [2]:
# Refer to pystac-client docs:
# https://pystac-client.readthedocs.io/en/stable/usage.html

retry = Retry(
    total=5, backoff_factor=1, status_forcelist=[502, 503, 504], allowed_methods=None
)
stac_api_io = StacApiIO(max_retries=retry)

catalog = Client.open(
    "http://localhost:8000", stac_io=stac_api_io
)

# Checking if the API supports item searching
item_search = catalog.conforms_to("ITEM_SEARCH")
print("Item search:", item_search)

# Get all available collections in STAC API
for collection in catalog.get_all_collections():
    print(collection.id)

Item search: True
canari
icenet_north
icenet_south


In [ ]:
class STAC:
    def __init__(self, STAC_FASTAPI_URL: str) -> None:
        self._url = STAC_FASTAPI_URL
        self._catalog = Client.open(STAC_FASTAPI_URL)

    def _search_collection(self, collection_id) -> ItemSearch:
        search = self._catalog.search(collections=[collection_id], max_items=None)
        return search

    def _search_item(
        self, collection_id, item_id, max_items: int | None = None
    ) -> ItemSearch:
        search = self._catalog.search(
            collections=[collection_id], ids=item_id, max_items=max_items
        )
        return search

    def get_catalog_collection_ids(
        self, resolve: bool = False
    ) -> Iterable[Collection] | tuple[Collection]:
        # Get all available collections in STAC API
        collections = self._catalog.get_all_collections()
        return tuple(collections) if resolve else collections

    def get_collection_items(self, collection_id, resolve: bool = False):
        collection = self._catalog.get_collection(collection_id)
        items = collection.get_items()
        return tuple(items) if resolve else items

    def get_collection_extents(self, collection_id):
        collection = self._catalog.get_collection(collection_id)
        print(collection)
        temporal_extent = collection.extent.temporal.intervals[0]
        spatial_extent = collection.extent.spatial.bboxes[0]
        return temporal_extent, spatial_extent

    def get_collection_forecast_init_dates(self, collection_id) -> list[dt]:
        items = self.get_collection_items(collection_id)
        datetimes = sorted(
            {item.datetime for item in items if item.datetime is not None}
        )
        return datetimes

    def get_item(self, collection_id, item_id) -> Item:
        # Using db likely to be faster than using get_collection_items, then filtering here in Python.
        search = self._search_item(collection_id, item_id)
        item = tuple(search.items())
        if len(item) > 1:
            raise ValueError(
                f"Multiple items with id {item_id} found within {collection_id} collection."
            )
        else:
            item = item[0]
        return item

    def get_item_properties(self, collection_id, item_id):
        item = self.get_item(collection_id, item_id)
        return item.properties

    def get_item_leadtime(self, collection_id, item_id) -> str:
        properties = self.get_item_properties(collection_id, item_id)
        return properties["forecast:leadtime"]

    def get_item_extents(self, collection_id, item_id):
        item = self.get_item(collection_id, item_id)
        item_props = item.properties
        temporal_extent = (
            item_props["forecast:reference_time"],
            item_props["forecast:end_time"],
        )
        temporal_extent = [
            parser.isoparse(iso_string) for iso_string in temporal_extent
        ]
        # Convert to match datetime like `get_collection_extents`.
        spatial_extent = item.bbox
        return temporal_extent, spatial_extent

    def get_item_cogs(self, collection_id, item_id):
        item = self.get_item(collection_id, item_id)
        assets = item.get_assets(media_type=MediaType.COG, role="data")
        return assets


In [4]:
STAC_FASTAPI_URL = "http://localhost:8000"
collection_id = "icenet_north"

stac = STAC(STAC_FASTAPI_URL)
collections = stac.get_catalog_collection_ids(resolve=True)
items = stac.get_collection_items(collection_id, resolve=True)
forecast_init_dates = stac.get_collection_forecast_init_dates(collection_id)

item_id = items[0].id
leadtime = stac.get_item_leadtime(collection_id, item_id)
collection_extents = stac.get_collection_extents(collection_id)
item_extents = stac.get_item_extents(collection_id, item_id)


<CollectionClient id=icenet_north>


In [5]:
stac.get_item_cogs(collection_id, item_id)

{'2025-01-31 00:00': <Asset href=http://192.168.68.114:8001/data/cogs/icenet_north/2025-01-31/forecast_init_20250131_0000_lead_20250131_0000.tif>,
 '2025-02-01 00:00': <Asset href=http://192.168.68.114:8001/data/cogs/icenet_north/2025-01-31/forecast_init_20250131_0000_lead_20250201_0000.tif>,
 '2025-02-02 00:00': <Asset href=http://192.168.68.114:8001/data/cogs/icenet_north/2025-01-31/forecast_init_20250131_0000_lead_20250202_0000.tif>,
 '2025-02-03 00:00': <Asset href=http://192.168.68.114:8001/data/cogs/icenet_north/2025-01-31/forecast_init_20250131_0000_lead_20250203_0000.tif>,
 '2025-02-04 00:00': <Asset href=http://192.168.68.114:8001/data/cogs/icenet_north/2025-01-31/forecast_init_20250131_0000_lead_20250204_0000.tif>,
 '2025-02-05 00:00': <Asset href=http://192.168.68.114:8001/data/cogs/icenet_north/2025-01-31/forecast_init_20250131_0000_lead_20250205_0000.tif>,
 '2025-02-06 00:00': <Asset href=http://192.168.68.114:8001/data/cogs/icenet_north/2025-01-31/forecast_init_20250131_0

In [6]:
print("First item:", items[0].id)
print("Leadtime:", leadtime)
print("Collection extents:", collection_extents)
print("Item extents:", item_extents)

First item: forecast_init_20250131_0000
Leadtime: 93
Collection extents: ([datetime.datetime(2025, 1, 4, 0, 0, tzinfo=tzutc()), datetime.datetime(2025, 1, 31, 0, 0, tzinfo=tzutc())], [-180.0, 16.623926693003686, 180.0, 90.0])
Item extents: ([datetime.datetime(2025, 1, 31, 0, 0, tzinfo=tzutc()), datetime.datetime(2025, 5, 3, 0, 0, tzinfo=tzutc())], [-180.0, 16.623926693003686, 180.0, 90.0])


In [7]:
print("Forecast init date (type):", type(forecast_init_dates[0]))
print("Forecast init date (first)", forecast_init_dates[0])

Forecast init date (type): <class 'datetime.datetime'>
Forecast init date (first) 2025-01-04 00:00:00+00:00
